In [1]:
!pip install --quiet transformers
!pip install --quiet sentencepiece
!pip install --quiet datasets
!pip install --quiet rouge_score
! pip install --quiet  evaluate


[notice] A new release of pip is available: 23.3.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from datasets import load_dataset
import pandas as pd

languages = ["telugu", "urdu", "marathi", "hindi", "tamil", "bengali", "english"]

dfs = []
for lang in languages:
    dataset = load_dataset("csebuetnlp/xlsum", lang, split="train[:500]")
    df = dataset.to_pandas()
    dfs.append(df)

df_train = pd.concat(dfs, ignore_index=True)

print(df_train.shape)
print(df_train.head())

(3500, 5)
                       id                                                url  \
0  international-53649907  https://www.bbc.com/telugu/international-53649907   
1          india-46550604          https://www.bbc.com/telugu/india-46550604   
2          india-43404438          https://www.bbc.com/telugu/india-43404438   
3  international-54671956  https://www.bbc.com/telugu/international-54671956   
4                53723894                https://www.bbc.com/telugu/53723894   

                                               title  \
0  పాకిస్తాన్ ఎయిర్‌లైన్స్‌లో నకిలీ లైసెన్సుల పైల...   
1  తెలంగాణ ముఖ్యమంత్రిగా కేసీఆర్ రెండోసారి ప్రమాణ...   
2  ‘అధికారం కొన్ని కులాల గుప్పిట్లోనే ఉండాలా? కుద...   
3  పోలండ్‌లో కొత్త అబార్షన్ చట్టాలను వ్యతిరేకిస్త...   
4  దిల్లీ అల్లర్లపై పరస్పర విరుద్ధ నివేదికలు... ఏ...   

                                             summary  \
0  పాకిస్తాన్ విమానయాన రంగంలో కొత్త సంక్షోభం మొదల...   
1  తెలంగాణ ముఖ్యమంత్రిగా కల్వకుంట్ల చంద్రశేఖర్ రా...   
2  గ

In [3]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import AdamW, get_scheduler

import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

from tqdm.auto import tqdm

import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import evaluate

C:\Users\SRMAPCSELAB2022-147\AppData\Roaming\Python\Python39\site-packages\torch\utils\_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(


In [4]:
tokenizer = AutoTokenizer.from_pretrained("csebuetnlp/mT5_multilingual_XLSum")

model = AutoModelForSeq2SeqLM.from_pretrained("csebuetnlp/mT5_multilingual_XLSum")

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
C:\Users\SRMAPCSELAB2022-147\AppData\Roaming\Python\Python39\site-packages\transformers\convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [5]:
class SummaryDataset(Dataset):
    def __init__(
        self,
        data=df,
        tokenizer=tokenizer,
        text_max_token_len = 200,
        summary_max_token_len = 12
    ):
        self.tokenizer = tokenizer
        self.data = data
        self.text_max_token_len = text_max_token_len
        self.summary_max_token_len = summary_max_token_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index: int):
        data_row = self.data.iloc[index]

        text = data_row['text']

        text_encoding = tokenizer(
            text,
            max_length=self.text_max_token_len,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            add_special_tokens=True,
            return_tensors='pt'
        )

        summary_encoding = tokenizer(
            data_row['summary'],
            max_length=self.summary_max_token_len,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            add_special_tokens=True,
            return_tensors='pt'
        )

        labels = summary_encoding['input_ids']
        labels[labels == tokenizer.pad_token_id] = -100

        return dict(
            input_ids=text_encoding['input_ids'].flatten(),
            attention_mask=text_encoding['attention_mask'].flatten(),
            labels=labels.flatten(),
            decoder_attention_mask=summary_encoding['attention_mask'].flatten()
        )

In [6]:
df_train, df_test = train_test_split(df, test_size=0.2, random_state=42)

train_dataset = SummaryDataset(data=df_train)
test_dataset = SummaryDataset(data=df_test)

train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=10)
eval_dataloader = DataLoader(test_dataset, batch_size=10)

In [7]:
num_epochs = 50

num_training_steps = num_epochs * len(train_dataloader)

optimizer = AdamW(model.parameters())
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps
)

progress_bar = tqdm(range(num_training_steps))

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}

        outputs = model(**batch)
#         logits = outputs.logits
#         predictions = torch.argmax(logits, dim=-1)
#         print(predictions)
#         print(batch["labels"])

        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()

        optimizer.zero_grad()
        progress_bar.update()

    torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss,
            }, f'./t5-multi.pth')

    print(f'epoch: {epoch + 1} -- loss: {loss}')

  0%|          | 0/2000 [00:00<?, ?it/s]

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


epoch: 1 -- loss: 3.2762844562530518
epoch: 2 -- loss: 1.5421720743179321
epoch: 3 -- loss: 0.5501308441162109
epoch: 4 -- loss: 0.4858549237251282
epoch: 5 -- loss: 0.2868013083934784
epoch: 6 -- loss: 0.3049148619174957
epoch: 7 -- loss: 0.3341846168041229
epoch: 8 -- loss: 0.18032243847846985
epoch: 9 -- loss: 0.35358351469039917
epoch: 10 -- loss: 0.053530506789684296
epoch: 11 -- loss: 0.0654199942946434
epoch: 12 -- loss: 0.029196597635746002
epoch: 13 -- loss: 0.1069263219833374
epoch: 14 -- loss: 0.1366676688194275
epoch: 15 -- loss: 0.010739903897047043
epoch: 16 -- loss: 0.06059781461954117
epoch: 17 -- loss: 0.0054414705373346806
epoch: 18 -- loss: 0.00869010016322136
epoch: 19 -- loss: 0.019023699685931206
epoch: 20 -- loss: 0.006694440729916096
epoch: 21 -- loss: 0.010097550228238106
epoch: 22 -- loss: 0.015266850590705872
epoch: 23 -- loss: 0.0028011088725179434
epoch: 24 -- loss: 0.005589862819761038
epoch: 25 -- loss: 0.09538896381855011
epoch: 26 -- loss: 0.00203347834

In [8]:
import torch
import evaluate

rouge = evaluate.load("rouge")
bleu = evaluate.load("bleu")

model.eval()

all_predictions = []
all_references = []

for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}

    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)

    decoded_predictions = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_references = tokenizer.batch_decode(batch["labels"], skip_special_tokens=True)

    all_predictions.extend(decoded_predictions)
    all_references.extend(decoded_references)

rouge_score = rouge.compute(predictions=all_predictions, references=all_references)
bleu_score = bleu.compute(predictions=all_predictions, references=all_references)

print("ROUGE Score:", rouge_score)
print("BLEU Score:", bleu_score)


ROUGE Score: {'rouge1': 0.27861247395535316, 'rouge2': 0.08354190417425712, 'rougeL': 0.272987796259468, 'rougeLsum': 0.2714781499460291}
BLEU Score: {'bleu': 0.07635851240766857, 'precisions': [0.2927536231884058, 0.09830508474576272, 0.061224489795918366, 0.038461538461538464], 'brevity_penalty': 0.8415887806065058, 'length_ratio': 0.8529048207663782, 'translation_length': 690, 'reference_length': 809}
